# Population Fitness Evaluation

Evaluates multiple neural network individuals on a single trading epoch. Demonstrates population-level analysis and prepares for genetic algorithm operations.

## 🔧 Configuration

**Modify these parameters to experiment with different setups:**

In [ ]:
# 🔧 CONFIGURATION - Modify these network
POPULATION_SIZE = 100
LAYER_SIZES = [1, 100, 200, 3]  # Input -> Hidden1 -> Hidden2 -> Output
ACTIVATIONS = [1, 1, 2]         # [ReLU, ReLU, Sigmoid]
EPOCH_ID = 1                    # Which epoch to analyze
RANDOM_SEED = 42                # For reproducibility

# 📝 Architecture Options:
# Small:  [1, 20, 10, 3]
# Medium: [1, 50, 100, 3] 
# Large:  [1, 200, 400, 200, 3]
# Deep:   [1, 50, 50, 50, 50, 3]

# 🎯 Activation Functions:
# 0 = Linear
# 1 = ReLU
# 2 = Sigmoid  
# 3 = Tanh

print(f"📊 Configuration:")
print(f"   Population size: {POPULATION_SIZE}")
print(f"   Architecture: {' -> '.join(map(str, LAYER_SIZES))}")
print(f"   Activations: {ACTIVATIONS}")
print(f"   Target epoch: {EPOCH_ID}")
print(f"   Random seed: {RANDOM_SEED}")

In [ ]:
import polars as pl
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import time
import sys
sys.path.append('../../')
import numba_ga

# Import population fitness functions
sys.path.append('../../src/fitnesses')
from fitness import evaluate_population_fitness_relative, evaluate_individual_fitness_relative

## 1. Architecture Analysis

In [ ]:
# Convert to numpy arrays for numba compatibility
layer_sizes = np.array(LAYER_SIZES, dtype=np.int64)
activations = np.array(ACTIVATIONS, dtype=np.int64)

# Calculate architecture properties
total_params = numba_ga.get_total_parameters(layer_sizes)
activation_names = [numba_ga.get_activation_name(a) for a in activations]

print(f"🧠 Network Architecture Analysis:")
print(f"   Layers: {len(layer_sizes)}")
print(f"   Architecture: {' -> '.join(map(str, layer_sizes))}")
print(f"   Activations: {' -> '.join(activation_names)}")
print(f"   Total parameters: {total_params:,}")
print(f"   Memory per individual: ~{total_params * 8 / 1024:.1f} KB")
print(f"   Population memory: ~{total_params * 8 * POPULATION_SIZE / (1024*1024):.1f} MB")

# Estimate complexity
complexity_score = total_params * POPULATION_SIZE
if complexity_score < 100_000:
    complexity = "Low - Very fast evaluation"
elif complexity_score < 1_000_000:
    complexity = "Medium - Fast evaluation"
elif complexity_score < 10_000_000:
    complexity = "High - Moderate evaluation time"
else:
    complexity = "Very High - Slower evaluation"

print(f"   Complexity: {complexity}")
print(f"   Complexity score: {complexity_score:,}")

## 2. Data Loading & Epoch Selection

In [ ]:
# Load processed data with epochs
df = pl.read_parquet('../../data/mock_processed.parquet')

print(f"📊 Dataset Overview:")
print(f"   Total shape: {df.shape}")
print(f"   Available epochs: {df['epoch_id'].min()} - {df['epoch_id'].max()}")

# Extract target epoch
epoch_data = df.filter(pl.col('epoch_id') == EPOCH_ID).sort('timestamp')

if len(epoch_data) == 0:
    print(f"❌ Error: Epoch {EPOCH_ID} not found. Available epochs: {sorted(df['epoch_id'].unique().to_list())}")
    raise ValueError(f"Invalid epoch ID: {EPOCH_ID}")

print(f"\n🕐 Epoch {EPOCH_ID} Characteristics:")
print(f"   Number of ticks: {len(epoch_data):,}")
print(f"   Time range: {epoch_data['timestamp'].min():.1f} - {epoch_data['timestamp'].max():.1f} seconds")
print(f"   Duration: {(epoch_data['timestamp'].max() - epoch_data['timestamp'].min()) / 60:.1f} minutes")
print(f"   Price range: ${epoch_data['price'].min():.2f} - ${epoch_data['price'].max():.2f}")
print(f"   Price change: {((epoch_data['price'][-1] - epoch_data['price'][0]) / epoch_data['price'][0] * 100):.2f}%")

In [ ]:
# Visualize the selected epoch
timestamps = epoch_data['timestamp'].to_numpy()
prices = epoch_data['price'].to_numpy()
time_minutes = (timestamps - timestamps[0]) / 60

fig = go.Figure()

fig.add_trace(
    go.Scatter(
        x=time_minutes,
        y=prices,
        mode='lines',
        name=f'Epoch {EPOCH_ID} Price',
        line=dict(color='blue', width=1.5)
    )
)

fig.update_layout(
    title=f"Epoch {EPOCH_ID}: Price Time Series ({len(epoch_data):,} ticks over {time_minutes[-1]:.1f} minutes)",
    xaxis_title="Time (minutes from epoch start)",
    yaxis_title="Price ($)",
    width=800, height=400,
    hovermode='x unified'
)

fig.show()

print(f"📈 Trading Environment:")
print(f"   Starting price: ${prices[0]:.2f}")
print(f"   Ending price: ${prices[-1]:.2f}")
print(f"   Buy-and-hold return: {((prices[-1] - prices[0]) / prices[0] * 100):.2f}%")
print(f"   Price volatility: {np.std(prices):.2f}")
print(f"   Number of opportunities: {len(prices):,} decision points")

## 3. Population Initialization

In [ ]:
print(f"🧬 Initializing population of {POPULATION_SIZE} individuals...")

# Initialize population with configured network
population = numba_ga.initialize_population(POPULATION_SIZE, layer_sizes, seed=RANDOM_SEED)

print(f"\n👥 Population Properties:")
print(f"   Shape: {population.shape}")
print(f"   Memory usage: {population.nbytes / (1024*1024):.2f} MB")
print(f"   Data type: {population.dtype}")

# Population diversity analysis
all_weights = population.flatten()
print(f"\n📊 Weight Distribution Across Population:")
print(f"   Total weights: {len(all_weights):,}")
print(f"   Mean: {np.mean(all_weights):.6f}")
print(f"   Std: {np.std(all_weights):.6f}")
print(f"   Range: {np.min(all_weights):.3f} to {np.max(all_weights):.3f}")

# Check population diversity (no identical individuals)
unique_individuals = len(np.unique(population, axis=0))
print(f"\n🎲 Population Diversity:")
print(f"   Unique individuals: {unique_individuals}/{POPULATION_SIZE}")
print(f"   Diversity: {'Perfect ✅' if unique_individuals == POPULATION_SIZE else 'Duplicates found ❌'}")

# Sample individual analysis
sample_individual = population[0]
print(f"\n🔍 Sample Individual (ID 0):")
print(f"   Parameters: {len(sample_individual):,}")
print(f"   Weight range: {np.min(sample_individual):.3f} to {np.max(sample_individual):.3f}")
print(f"   Mean: {np.mean(sample_individual):.6f}")
print(f"   Std: {np.std(sample_individual):.6f}")

## 4. Population Fitness Evaluation

In [ ]:
print(f"🚀 Evaluating {POPULATION_SIZE} individuals on Epoch {EPOCH_ID}...")
print(f"   Processing {len(timestamps):,} time steps per individual")
print(f"   Total evaluations: {POPULATION_SIZE * len(timestamps):,}")

# Warm up the JIT compilation
print(f"\n⚡ Warming up JIT compilation...")
small_pop = population[:5]  # Use small subset for warmup
_ = evaluate_population_fitness_relative(small_pop, layer_sizes, activations, timestamps, prices)
print(f"   JIT compilation complete ✅")

# Full population evaluation with timing
print(f"\n📊 Running full population evaluation...")
start_time = time.time()

fitness_scores = evaluate_population_fitness_relative(
    population, layer_sizes, activations, timestamps, prices
)

evaluation_time = time.time() - start_time

print(f"\n⏱️ Performance Results:")
print(f"   Evaluation time: {evaluation_time:.3f} seconds")
print(f"   Individuals per second: {POPULATION_SIZE / evaluation_time:.1f}")
print(f"   Time per individual: {evaluation_time / POPULATION_SIZE * 1000:.2f} ms")
print(f"   Evaluations per second: {(POPULATION_SIZE * len(timestamps)) / evaluation_time:,.0f}")

print(f"\n💰 Fitness Results:")
print(f"   Fitness array shape: {fitness_scores.shape}")
print(f"   Best fitness: {np.max(fitness_scores):.4f}")
print(f"   Worst fitness: {np.min(fitness_scores):.4f}")
print(f"   Mean fitness: {np.mean(fitness_scores):.4f}")
print(f"   Median fitness: {np.median(fitness_scores):.4f}")
print(f"   Std deviation: {np.std(fitness_scores):.4f}")

# Convert to returns for easier interpretation
returns = (fitness_scores - 1.0) * 100
print(f"\n📈 Return Analysis:")
print(f"   Best return: {np.max(returns):.2f}%")
print(f"   Worst return: {np.min(returns):.2f}%")
print(f"   Mean return: {np.mean(returns):.2f}%")
print(f"   Profitable individuals: {np.sum(returns > 0)}/{POPULATION_SIZE} ({np.sum(returns > 0)/POPULATION_SIZE*100:.1f}%)")

# Buy-and-hold benchmark
buy_hold_return = ((prices[-1] - prices[0]) / prices[0]) * 100
beat_market = np.sum(returns > buy_hold_return)
print(f"   Beat buy-and-hold ({buy_hold_return:.2f}%): {beat_market}/{POPULATION_SIZE} ({beat_market/POPULATION_SIZE*100:.1f}%)")

## 5. Fitness Distribution Analysis

In [ ]:
# Create fitness distribution visualization
fig = make_subplots(
    rows=2, cols=2,
    subplot_titles=[
        'Fitness Distribution (Portfolio Values)',
        'Return Distribution (%)',
        'Fitness vs Individual ID',
        'Top 10 vs Bottom 10 Performers'
    ],
    vertical_spacing=0.12,
    horizontal_spacing=0.1
)

# 1. Fitness histogram
fig.add_trace(
    go.Histogram(
        x=fitness_scores,
        nbinsx=30,
        name='Fitness',
        marker_color='blue',
        opacity=0.7
    ),
    row=1, col=1
)

# Add vertical line at 1.0 (break-even)
fig.add_vline(x=1.0, line_dash="dash", line_color="red", row=1, col=1)

# 2. Returns histogram
fig.add_trace(
    go.Histogram(
        x=returns,
        nbinsx=30,
        name='Returns',
        marker_color='green',
        opacity=0.7
    ),
    row=1, col=2
)

# Add vertical lines at 0% and buy-hold return
fig.add_vline(x=0, line_dash="dash", line_color="red", row=1, col=2)
fig.add_vline(x=buy_hold_return, line_dash="dot", line_color="orange", row=1, col=2)

# 3. Fitness scatter plot
individual_ids = np.arange(POPULATION_SIZE)
fig.add_trace(
    go.Scatter(
        x=individual_ids,
        y=fitness_scores,
        mode='markers',
        name='Individual Fitness',
        marker=dict(color=fitness_scores, colorscale='RdYlGn', size=4)
    ),
    row=2, col=1
)

# Add horizontal line at 1.0
fig.add_hline(y=1.0, line_dash="dash", line_color="red", row=2, col=1)

# 4. Top vs Bottom performers
sorted_indices = np.argsort(fitness_scores)
top_10 = sorted_indices[-10:]
bottom_10 = sorted_indices[:10]

fig.add_trace(
    go.Scatter(
        x=list(range(10)),
        y=fitness_scores[bottom_10],
        mode='markers+lines',
        name='Bottom 10',
        marker=dict(color='red', size=6),
        line=dict(color='red')
    ),
    row=2, col=2
)

fig.add_trace(
    go.Scatter(
        x=list(range(10)),
        y=fitness_scores[top_10],
        mode='markers+lines',
        name='Top 10',
        marker=dict(color='green', size=6),
        line=dict(color='green')
    ),
    row=2, col=2
)

# Update layout
fig.update_xaxes(title_text="Portfolio Value", row=1, col=1)
fig.update_xaxes(title_text="Return (%)", row=1, col=2)
fig.update_xaxes(title_text="Individual ID", row=2, col=1)
fig.update_xaxes(title_text="Rank", row=2, col=2)

fig.update_yaxes(title_text="Count", row=1, col=1)
fig.update_yaxes(title_text="Count", row=1, col=2)
fig.update_yaxes(title_text="Portfolio Value", row=2, col=1)
fig.update_yaxes(title_text="Portfolio Value", row=2, col=2)

fig.update_layout(
    title=f"Population Fitness Analysis - {POPULATION_SIZE} Individuals on Epoch {EPOCH_ID}",
    height=800,
    width=1000,
    showlegend=True
)

fig.show()

## 6. Individual Behavior Analysis

In [ ]:
# Analyze top and bottom performers in detail
sorted_indices = np.argsort(fitness_scores)
best_individual_idx = sorted_indices[-1]
worst_individual_idx = sorted_indices[0]
median_individual_idx = sorted_indices[len(sorted_indices)//2]

print(f"🏆 Performance Rankings:")
print(f"   Best individual: #{best_individual_idx} (fitness: {fitness_scores[best_individual_idx]:.4f}, return: {(fitness_scores[best_individual_idx]-1)*100:.2f}%)")
print(f"   Median individual: #{median_individual_idx} (fitness: {fitness_scores[median_individual_idx]:.4f}, return: {(fitness_scores[median_individual_idx]-1)*100:.2f}%)")
print(f"   Worst individual: #{worst_individual_idx} (fitness: {fitness_scores[worst_individual_idx]:.4f}, return: {(fitness_scores[worst_individual_idx]-1)*100:.2f}%)")

# Function to analyze individual trading behavior
from numba import njit

@njit
def analyze_individual_behavior(parameters, layer_sizes, activations, timestamps, prices):
    """Returns detailed trading analysis for visualization."""
    # Pre-compute shared data
    param_indices, neuron_indices = numba_ga.compute_layer_indices(layer_sizes)
    prices_normalized = prices / prices[0]
    
    # Arrays to track behavior
    actions = np.zeros(len(timestamps), dtype=np.int64)
    network_outputs = np.zeros((len(timestamps), 3), dtype=np.float64)
    portfolio_history = np.zeros(len(timestamps), dtype=np.float64)
    
    # Network state
    input_buffer = np.zeros(1, dtype=np.float64)
    current_states = np.zeros(np.sum(layer_sizes[1:]), dtype=np.float64)
    current_time = 0.0
    
    # Trading state
    position = 0
    buy_price_norm = 0.0
    portfolio_value = 1.0
    
    # Main loop
    for i in range(len(timestamps)):
        input_buffer[0] = prices_normalized[i]
        inputs = (timestamps[i], input_buffer)
        
        output, new_states, new_time = numba_ga.predict_individual(
            parameters, layer_sizes, activations, inputs,
            current_states, current_time, param_indices, neuron_indices
        )
        
        network_outputs[i] = output
        
        action = (0 if output[0] >= output[1] and output[0] >= output[2] 
                 else 1 if output[1] >= output[2] else 2)
        
        # Trading logic
        if position == 0 and action == 2:
            position = 1
            buy_price_norm = prices_normalized[i]
            actual_action = 2  # BUY
        elif position == 1 and action == 0:
            sell_price_norm = prices_normalized[i]
            portfolio_value *= (sell_price_norm / buy_price_norm)
            position = 0
            actual_action = 0  # SELL
        else:
            actual_action = 1  # HOLD
        
        actions[i] = actual_action
        portfolio_history[i] = portfolio_value
        
        current_states = new_states
        current_time = new_time
    
    # Close position if holding at end
    if position == 1:
        final_price_norm = prices_normalized[-1]
        portfolio_value *= (final_price_norm / buy_price_norm)
        portfolio_history[-1] = portfolio_value
    
    return actions, network_outputs, portfolio_history

# Analyze the three key individuals
individuals_to_analyze = [
    (best_individual_idx, "Best", "green"),
    (median_individual_idx, "Median", "orange"), 
    (worst_individual_idx, "Worst", "red")
]

analysis_results = {}
for idx, label, color in individuals_to_analyze:
    actions, outputs, portfolio = analyze_individual_behavior(
        population[idx], layer_sizes, activations, timestamps, prices
    )
    analysis_results[label] = {
        'idx': idx,
        'actions': actions,
        'outputs': outputs, 
        'portfolio': portfolio,
        'color': color,
        'fitness': fitness_scores[idx]
    }

print(f"\n🔍 Behavioral Analysis Complete")
for label, data in analysis_results.items():
    action_counts = np.bincount(data['actions'], minlength=3)
    num_trades = action_counts[0] + action_counts[2]
    print(f"   {label} (#{data['idx']}): {num_trades} trades, final value: {data['fitness']:.4f}")
    print(f"     Actions: {action_counts[0]} sells, {action_counts[1]} holds, {action_counts[2]} buys")

In [ ]:
# Visualize individual behaviors
fig = make_subplots(
    rows=4, cols=1,
    subplot_titles=[
        'Price Evolution with Trading Actions',
        'Network Outputs (Sell/Hold/Buy Signals)',
        'Actual Actions Taken',
        'Portfolio Value Evolution'
    ],
    vertical_spacing=0.08,
    specs=[[{"secondary_y": False}]] * 4
)

time_minutes = (timestamps - timestamps[0]) / 60

# 1. Price with trading markers
fig.add_trace(
    go.Scatter(
        x=time_minutes,
        y=prices,
        mode='lines',
        name='Price',
        line=dict(color='blue', width=1.5),
        opacity=0.8
    ),
    row=1, col=1
)

# Add trading actions for each individual
for label, data in analysis_results.items():
    buy_mask = data['actions'] == 2
    sell_mask = data['actions'] == 0
    
    if np.any(buy_mask):
        fig.add_trace(
            go.Scatter(
                x=time_minutes[buy_mask],
                y=prices[buy_mask],
                mode='markers',
                name=f'{label} BUY',
                marker=dict(color=data['color'], size=6, symbol='triangle-up')
            ),
            row=1, col=1
        )
    
    if np.any(sell_mask):
        fig.add_trace(
            go.Scatter(
                x=time_minutes[sell_mask],
                y=prices[sell_mask],
                mode='markers',
                name=f'{label} SELL',
                marker=dict(color=data['color'], size=6, symbol='triangle-down')
            ),
            row=1, col=1
        )

# 2. Network outputs (show only best individual to avoid clutter)
best_data = analysis_results['Best']
output_names = ['Sell Signal', 'Hold Signal', 'Buy Signal']
output_colors = ['red', 'gray', 'green']

for i in range(3):
    fig.add_trace(
        go.Scatter(
            x=time_minutes,
            y=best_data['outputs'][:, i],
            mode='lines',
            name=f'Best: {output_names[i]}',
            line=dict(color=output_colors[i], width=1),
            opacity=0.7
        ),
        row=2, col=1
    )

# 3. Actions comparison
for label, data in analysis_results.items():
    fig.add_trace(
        go.Scatter(
            x=time_minutes,
            y=data['actions'],
            mode='markers',
            name=f'{label} Actions',
            marker=dict(color=data['color'], size=2),
            opacity=0.6
        ),
        row=3, col=1
    )

# 4. Portfolio evolution
for label, data in analysis_results.items():
    fig.add_trace(
        go.Scatter(
            x=time_minutes,
            y=data['portfolio'],
            mode='lines',
            name=f'{label} Portfolio',
            line=dict(color=data['color'], width=2)
        ),
        row=4, col=1
    )

# Add break-even line
fig.add_hline(y=1.0, line_dash="dash", line_color="black", opacity=0.5, row=4, col=1)

# Update layout
fig.update_xaxes(title_text="Time (minutes)", row=4, col=1)
fig.update_yaxes(title_text="Price ($)", row=1, col=1)
fig.update_yaxes(title_text="Output Value", row=2, col=1)
fig.update_yaxes(title_text="Action", row=3, col=1, tickvals=[0, 1, 2], ticktext=['SELL', 'HOLD', 'BUY'])
fig.update_yaxes(title_text="Portfolio Value", row=4, col=1)

fig.update_layout(
    title=f"Individual Behavior Comparison - Best vs Median vs Worst Performers",
    height=1200,
    width=1000,
    showlegend=True
)

fig.show()

## 7. Performance Scaling Analysis

In [ ]:
# Test different population sizes to show scaling
test_sizes = [10, 25, 50, 100, 200] if POPULATION_SIZE >= 200 else [10, 25, 50, 100]
if POPULATION_SIZE not in test_sizes:
    test_sizes.append(POPULATION_SIZE)
test_sizes = sorted(test_sizes)

print(f"⚡ Performance Scaling Analysis")
print(f"   Testing population sizes: {test_sizes}")
print(f"   Architecture: {' -> '.join(map(str, LAYER_SIZES))}")

scaling_results = []

for size in test_sizes:
    if size <= POPULATION_SIZE:
        # Use subset of current population
        test_pop = population[:size]
    else:
        # Generate larger population
        test_pop = numba_ga.initialize_population(size, layer_sizes, seed=RANDOM_SEED)
    
    # Warm up for this size (first run always slower)
    _ = evaluate_population_fitness_relative(test_pop[:min(5, size)], layer_sizes, activations, timestamps, prices)
    
    # Time the evaluation
    start_time = time.time()
    test_fitness = evaluate_population_fitness_relative(test_pop, layer_sizes, activations, timestamps, prices)
    eval_time = time.time() - start_time
    
    # Calculate metrics
    individuals_per_sec = size / eval_time
    time_per_individual = eval_time / size * 1000  # milliseconds
    
    scaling_results.append({
        'size': size,
        'time': eval_time,
        'ind_per_sec': individuals_per_sec,
        'ms_per_ind': time_per_individual,
        'best_fitness': np.max(test_fitness),
        'mean_fitness': np.mean(test_fitness)
    })
    
    print(f"   Size {size:3d}: {eval_time:.3f}s ({individuals_per_sec:.1f} ind/s, {time_per_individual:.2f} ms/ind)")

# Analyze scaling efficiency
print(f"\n📊 Scaling Efficiency:")
baseline_time_per_ind = scaling_results[0]['ms_per_ind']
for result in scaling_results:
    efficiency = baseline_time_per_ind / result['ms_per_ind'] * 100
    print(f"   Size {result['size']:3d}: {efficiency:.1f}% efficient (vs size {test_sizes[0]})")

# Memory scaling
print(f"\n💾 Memory Usage Scaling:")
for result in scaling_results:
    memory_mb = total_params * 8 * result['size'] / (1024*1024)
    print(f"   Size {result['size']:3d}: ~{memory_mb:.1f} MB")

In [ ]:
# Visualize scaling performance
sizes = [r['size'] for r in scaling_results]
times = [r['time'] for r in scaling_results]
ind_per_sec = [r['ind_per_sec'] for r in scaling_results]

fig = make_subplots(
    rows=1, cols=2,
    subplot_titles=['Evaluation Time vs Population Size', 'Throughput vs Population Size']
)

# Linear and ideal scaling reference lines
ideal_times = [times[0] * (s / sizes[0]) for s in sizes]
ideal_throughput = [ind_per_sec[0] for _ in sizes]  # Should stay constant

# Plot 1: Evaluation time
fig.add_trace(
    go.Scatter(
        x=sizes,
        y=times,
        mode='markers+lines',
        name='Actual Time',
        marker=dict(color='blue', size=8),
        line=dict(color='blue')
    ),
    row=1, col=1
)

fig.add_trace(
    go.Scatter(
        x=sizes,
        y=ideal_times,
        mode='lines',
        name='Ideal Linear',
        line=dict(color='gray', dash='dash')
    ),
    row=1, col=1
)

# Plot 2: Throughput
fig.add_trace(
    go.Scatter(
        x=sizes,
        y=ind_per_sec,
        mode='markers+lines',
        name='Actual Throughput',
        marker=dict(color='green', size=8),
        line=dict(color='green')
    ),
    row=1, col=2
)

fig.add_trace(
    go.Scatter(
        x=sizes,
        y=ideal_throughput,
        mode='lines',
        name='Ideal Constant',
        line=dict(color='gray', dash='dash')
    ),
    row=1, col=2
)

fig.update_xaxes(title_text="Population Size", row=1, col=1)
fig.update_xaxes(title_text="Population Size", row=1, col=2)
fig.update_yaxes(title_text="Time (seconds)", row=1, col=1)
fig.update_yaxes(title_text="Individuals/Second", row=1, col=2)

fig.update_layout(
    title=f"Population Fitness Evaluation - Performance Scaling",
    height=500,
    width=1000,
    showlegend=True
)

fig.show()

# Calculate parallel efficiency
if len(scaling_results) >= 2:
    parallel_efficiency = (scaling_results[0]['ind_per_sec'] / scaling_results[-1]['ind_per_sec']) * 100
    print(f"\n⚡ Parallel Efficiency: {parallel_efficiency:.1f}%")
    print(f"   (Higher is better - 100% means perfect scaling)")
    
    if parallel_efficiency > 90:
        print(f"   🎯 Excellent scaling - Numba parallel optimization working well!")
    elif parallel_efficiency > 70:
        print(f"   ✅ Good scaling - Minor overhead but efficient")
    else:
        print(f"   ⚠️ Scaling issues - May need optimization for large populations")

## 8. Summary & Next Steps

In [ ]:
print(f"📊 POPULATION FITNESS EVALUATION SUMMARY")
print(f"=" * 50)

print(f"\n🔧 Configuration Used:")
print(f"   Population size: {POPULATION_SIZE}")
print(f"   Architecture: {' -> '.join(map(str, LAYER_SIZES))}")
print(f"   Activations: {' -> '.join([numba_ga.get_activation_name(a) for a in activations])}")
print(f"   Total parameters: {total_params:,}")
print(f"   Epoch analyzed: {EPOCH_ID}")

print(f"\n💰 Financial Results:")
print(f"   Best performer: {np.max(returns):.2f}% return")
print(f"   Worst performer: {np.min(returns):.2f}% return")
print(f"   Population mean: {np.mean(returns):.2f}% return")
print(f"   Profitable traders: {np.sum(returns > 0)}/{POPULATION_SIZE} ({np.sum(returns > 0)/POPULATION_SIZE*100:.1f}%)")
print(f"   Beat market: {beat_market}/{POPULATION_SIZE} ({beat_market/POPULATION_SIZE*100:.1f}%)")
print(f"   Market return: {buy_hold_return:.2f}%")

print(f"\n⚡ Performance Metrics:")
print(f"   Evaluation time: {evaluation_time:.3f} seconds")
print(f"   Throughput: {POPULATION_SIZE / evaluation_time:.1f} individuals/second")
print(f"   Time per individual: {evaluation_time / POPULATION_SIZE * 1000:.2f} ms")
print(f"   Total decisions: {POPULATION_SIZE * len(timestamps):,}")

print(f"\n🧬 Population Insights:")
fitness_range = np.max(fitness_scores) - np.min(fitness_scores)
print(f"   Fitness diversity: {fitness_range:.4f} (range of outcomes)")
print(f"   Std deviation: {np.std(fitness_scores):.4f}")
print(f"   Perfect diversity: {'✅' if unique_individuals == POPULATION_SIZE else '❌'}")

# Trading behavior summary
total_actions = {}
for label, data in analysis_results.items():
    action_counts = np.bincount(data['actions'], minlength=3)
    total_actions[label] = action_counts

print(f"\n🎯 Trading Behavior (Top 3 Analyzed):")
for label, counts in total_actions.items():
    total_trades = counts[0] + counts[2]
    activity_rate = total_trades / len(timestamps) * 100
    print(f"   {label}: {total_trades} trades ({activity_rate:.2f}% activity rate)")

print(f"\n🚀 Ready for Genetic Algorithm:")
print(f"   ✅ Population initialization working")
print(f"   ✅ Fitness evaluation optimized")
print(f"   ✅ Performance scaling verified")
print(f"   ✅ Individual behavior analysis ready")
print(f"   ✅ Fitness diversity for selection")

print(f"\n📈 Next Steps:")
print(f"   1. Multi-epoch evaluation (test consistency)")
print(f"   2. Tournament selection (choose parents)")
print(f"   3. Crossover operations (breed new individuals)")
print(f"   4. Mutation operations (add diversity)")
print(f"   5. Evolutionary loop (improve over generations)")

print(f"\n✅ POPULATION FITNESS EVALUATION COMPLETE!")
print(f"   🎯 Foundation established for genetic algorithm evolution")
print(f"   📊 Population shows diverse trading strategies")
print(f"   ⚡ Performance optimized for large-scale evolution")